In [ ]:
from google.colab import files
files.upload()

In [ ]:
%%capture
!pip install git+https://github.com/joshcarty/google-searchconsole
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import data_table
!pip install umap-learn
data_table.enable_dataframe_formatter()

In [ ]:
import searchconsole
account = searchconsole.authenticate(client_config='client_secret_.json',serialize='credentials.json', flow='console')

In [ ]:
property_name = input('Insert the name of your website as listed in GSC: ')
webproperty=account[str(property_name)]

In [ ]:
def extract_gsc_data(webproperty, start, stop, *args):
 if webproperty is not None:
   print(f'Extracting data for {webproperty}')
   gsc_data = webproperty.query.range(start, stop).dimension(*args).get()
   return gsc_data
 else:
   print('Webproperty not found, please select the correct one')
   return None

In [ ]:
from datetime import datetime

ex = extract_gsc_data(webproperty, '2023-12-01', str(datetime.now().date()), 'query', 'page', 'date')

In [ ]:
df = pd.DataFrame(data=ex)
df.head()

In [ ]:
from google.colab import files
today_date = datetime.today().strftime('%Y-%m-%d')
filename = f'initialdataset_{today_date}.csv'
df.to_csv(filename, index=False)
files.download(filename)

In [ ]:
domain_name = str(input('Insert brand terms separated by a comma: ')).replace(',', '|')
import re
domain_name = re.sub(r"\s+", "", domain_name)
print('Remove all spaces using RegEx:\n')
df['Brand/Non-branded'] = np.where(
   df['query'].str.contains(domain_name), 'Brand', 'Non-branded'
)

In [ ]:
brand_count_df = df['Brand/Non-branded'].value_counts().rename_axis('category').to_frame('counts')
brand_count_df['Percentage'] = brand_count_df['counts']/sum(brand_count_df['counts'])
pd.options.display.float_format = '{:.2%}'.format
brand_count_df

In [ ]:
df.fillna(0, inplace = True)

In [ ]:
df['position'] = df['position'].round(0).astype('int64')

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

In [ ]:
ranking_queries = df.pivot_table(index=['position'], values=['query'], aggfunc=['count'])
ranking_queries.sort_values(by=['position']).head(10)

,count
,query
position,
1,44516
2,10096
3,12741
4,11878
5,9105
6,8685
7,6612
8,7472


In [ ]:
filename1 = f'ctr_avgposition_{today_date}.xlsx'
query_analysis.to_excel(filename1)
files.download(filename1)

In [ ]:
clicks_sum = df.groupby('date')['clicks'].sum()

In [ ]:
filename2 = f'clicks_month_{today_date}.xlsx'
clicks_sum.to_excel(filename2,index=False)
files.download(filename2)

In [ ]:
import seaborn as sns
sns.set_theme()

df_new = df.loc[(df['position'] <= 10) & (df['year'] != 2024),:]

df_heat = df_new.pivot_table(index = "position", columns = "month", values = "query", aggfunc='count')

f, ax = plt.subplots(figsize=(20, 12))

unique_months = sorted(df['date'].dt.strftime('%B').unique(), key=lambda x: datetime.strptime(x, '%B'))

x_axis_labels = unique_months

sns.heatmap(df_heat, annot=True, linewidths=.5, ax=ax, fmt='g', cmap = sns.cm.rocket_r, xticklabels=x_axis_labels)
ax.set(xlabel = 'Month', ylabel='Position', title = 'How query count per position changes with time')
plt.yticks(rotation=0)

In [ ]:
filename3 = f'heatmap_data_{today_date}.xlsx'
df_heat.to_excel(filename3,index=False)
files.download(filename3)

In [ ]:
import nltk

nltk.download('stopwords')
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

from sklearn.feature_extraction.text import CountVectorizer
c_vec = CountVectorizer(stop_words=stoplist, ngram_range=(2,3))
ngrams = c_vec.fit_transform(df['query'])
count_values = ngrams.toarray().sum(axis=0)
vocab = c_vec.vocabulary_
df_ngram = pd.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)
           ).rename(columns={0: 'frequency', 1:'bigram/trigram'})

df_ngram.head(20).style.background_gradient()

In [ ]:
filename4 = f'ngram_data_{today_date}.xlsx'
df_ngram.to_excel(filename4,index=False)
files.download(filename4)

In [ ]:
top_impressions = df[df['impressions'] >= df['impressions'].quantile(0.8)]
(top_impressions[top_impressions['ctr'] <= top_impressions['ctr'].quantile(0.2)].sort_values('impressions', ascending = False))

In [ ]:
filename5 = f'top_impressions_data{today_date}.xlsx'
top_impressions.to_excel(filename5,index=False)
files.download(filename5)

In [ ]:
!pip install mlxtend

from mlxtend.frequent_patterns import fpgrowth
from mlxtend.preprocessing import TransactionEncoder
from sklearn.preprocessing import LabelEncoder

In [ ]:
df['date'] = pd.to_datetime(df['date'])


In [ ]:
queries = df['query'].apply(lambda x: re.split(r'[“?^”<#(/\]=\-):[@÷\'&",’$.•⁄_+ ]', x))

te = TransactionEncoder()
te_ary = te.fit(queries).transform(queries)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = fpgrowth(df_encoded, min_support=0.01, use_colnames=True)

print(frequent_itemsets)

In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)

#print(rules)
filenamerules = f'associationrule{today_date}.xlsx'
rules.to_excel(filenamerules, index=False)
files.download(filenamerules)